In [1]:
import json
import re
from collections import defaultdict

ENTITY_KEYS = {
    "NUMBER", "SIZE", "STYLE", "TOPPING", "COMPLEX_TOPPING", "QUANTITY",
    "VOLUME", "DRINKTYPE", "CONTAINERTYPE"
}
ORDER_KEYS = {"PIZZAORDER", "DRINKORDER"}

def tokenize(s):
    tokens = re.findall(r'\(|\)|[^\s()]+', s)
    return tokens

def parse_tokens(tokens):
    stack = []
    current_list = []
    for token in tokens:
        if token == '(':
            stack.append(current_list)
            current_list = []
        elif token == ')':
            finished = current_list
            current_list = stack.pop()
            current_list.append(finished)
        else:
            current_list.append(token)
    return current_list
def extract_orders(structure, order_index=1):

    results = []

    if not isinstance(structure, list) or len(structure) == 0:
        return results, order_index

    first = structure[0]
    if isinstance(first, list):
        for elem in structure:
            sub_results, order_index = extract_orders(elem, order_index)
            results.extend(sub_results)
        return results, order_index

    if isinstance(first, str) and first in ORDER_KEYS:
        order_type = "PIZZAORDER" if first == "PIZZAORDER" else "DRINKORDER"
        current_order_sequence = order_index
        order_index += 1
        content_tokens = []
        for elem in structure[1:]:
            content_tokens.extend(collect_tokens(elem))
        for tok in content_tokens:
            results.append((tok, order_type, current_order_sequence))

        return results, order_index
    else:
        for elem in structure:
            sub_results, order_index = extract_orders(elem, order_index)
            results.extend(sub_results)
        return results, order_index
def collect_tokens(node):
    collected = []
    if isinstance(node, list):
        for sub in node:
            sub_tokens = collect_tokens(sub)
            collected.extend(sub_tokens)
    else:
        if node not in ["(", ")"] and not is_structural_key(node):
            collected.append(node)
    return collected

def is_structural_key(token):
    return token in [
        "ORDER","PIZZAORDER","DRINKORDER","NUMBER","SIZE","STYLE","TOPPING",
        "COMPLEX_TOPPING","QUANTITY","VOLUME","DRINKTYPE","CONTAINERTYPE","NOT"
    ]


In [2]:
def map_tokens_to_labels(input_text, reference_list):

    input_tokens = input_text.split()

    ref_index = 0
    ref_len = len(reference_list)

    output = []
    for token in input_tokens:
        if ref_index < ref_len:
            ref_token, ref_label, ref_seq = reference_list[ref_index]
            if token.lower() == ref_token.lower():
                output.append((token, ref_label, ref_seq))
                ref_index += 1
            else:
                # If not matched, assign O and None
                output.append((token, 'O', None))
        else:
            # No more reference tokens to match, so assign O and None
            output.append((token, 'O', None))

    return output

def label_input(input_text, top):
    tokens = tokenize(top)
    parsed = parse_tokens(tokens)

    order_info, _ = extract_orders(parsed)
    return map_tokens_to_labels(input_text, order_info)

top = "(ORDER i need (PIZZAORDER (NUMBER a ) (SIZE medium ) (TOPPING ham ) and (TOPPING pineapple ) pizza ) and (DRINKORDER (NUMBER a ) (VOLUME small ) (DRINKTYPE iced tea ) ) )"

input_text = "i need a medium ham and pineapple pizza and a small iced tea"
input_label_sequence = label_input(input_text, top)
print(input_label_sequence)

[('i', 'O', None), ('need', 'O', None), ('a', 'PIZZAORDER', 1), ('medium', 'PIZZAORDER', 1), ('ham', 'PIZZAORDER', 1), ('and', 'PIZZAORDER', 1), ('pineapple', 'PIZZAORDER', 1), ('pizza', 'PIZZAORDER', 1), ('and', 'O', None), ('a', 'DRINKORDER', 2), ('small', 'DRINKORDER', 2), ('iced', 'DRINKORDER', 2), ('tea', 'DRINKORDER', 2)]


In [3]:

def transform_to_labels(input_array):
    labeled_numbers = []

    for _, label, sequence in input_array:
        # Compute the numerical label
        if label == 'O' and sequence is None:
            numerical_label = 0  # Neutral/irrelevant
        elif label == 'PIZZAORDER':
            numerical_label = 10 + sequence  # Unique range for pizza orders
        elif label == 'DRINKORDER':
            numerical_label = 20 + sequence  # Unique range for drink orders
        else:
            numerical_label = 0  # Default fallback

        labeled_numbers.append(numerical_label)

    return labeled_numbers

transform_to_labels(input_label_sequence)

[0, 0, 11, 11, 11, 11, 11, 11, 0, 22, 22, 22, 22]

In [4]:
def transform_labeling(labels):
    label_map = {
        "B-PIZZAORDER": 1,
        "I-PIZZAORDER": 2,
        "B-DRINKORDER": 3,
        "I-DRINKORDER": 4,
        "O": 5
    }

    bio_labels = []
    seen_numbers = set()  # Track already encountered order numbers

    for label in labels:
        if label == 0:
            # Neutral/irrelevant token
            bio_labels.append(label_map["O"])
        else:
            # Determine if it's a PizzaOrder or DrinkOrder
            if 10 <= label < 20:
                label_type = "PIZZAORDER"
            elif 20 <= label < 30:
                label_type = "DRINKORDER"
            else:
                raise ValueError(f"Invalid label encountered: {label}")

            # Assign B- or I- based on first or repeated appearance
            if label not in seen_numbers:
                bio_label = label_map[f"B-{label_type}"]
                seen_numbers.add(label)
            else:
                bio_label = label_map[f"I-{label_type}"]

            bio_labels.append(bio_label)

    return bio_labels


In [5]:
def create_training_data(input_file: str, output_file: str):

    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            record = json.loads(line)
            src = record["test.SRC"]
            top = record["test.TOP"]

            labeled_input = label_input(src, top)
            numerical_labels = transform_to_labels(labeled_input)
            bio_labels = transform_labeling(numerical_labels)

            training_instance = {
                "text": src,
                "labels": bio_labels
            }

            outfile.write(json.dumps(training_instance) + "\n")

# File paths
input_file = "../dataset2/PIZZA_dev.json"
output_file = "../dataset2/dev_data_model1.json"

# Generate the training data
create_training_data(input_file,output_file)

In [6]:
import pandas as pd
train_path = '../dataset2/train_data_model1.json'
dev_path = "../dataset2/dev_data_model1.json"
df = pd.read_json(train_path, lines=True)
dev = pd.read_json(dev_path, lines=True)
df.describe()

,text,labels
count,1871720,1871720
unique,1800845,4688
top,i need four pies with pickle and broccoli,"[5, 5, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
freq,7,101642


In [7]:
dev.describe()

,text,labels
count,1357,1357
unique,1357,464
top,i want a pizza with pesto and mushrooms but no...,"[5, 5, 1, 2, 2, 2, 2, 2, 2, 2]"
freq,1,88


In [8]:
X_train = df['text']
y_train = df['labels']
X_test = dev['text']
y_test = dev['labels']

In [9]:
X_train[0], y_train[0]

('large pie with green pepper and with extra ham', [1, 2, 2, 2, 2, 2, 2, 2, 2])

In [10]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}  # Special tokens

def tokenize_output(output):
    """
    Tokenizes the structured output into meaningful tokens.
    Example:
        Input: "(ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"
        Output: ["(ORDER", "(PIZZAORDER", "(NUMBER", "a", "(SIZE", "large", "(TOPPING", "bbq", "pulled", "pork", ")", ")", ")", ")"]
    """
    tokens = re.findall(r"\(|\)|\w+|[^\s()]+", output)
    return tokens

def build_vocab(outputs, index):
    """
    Builds a vocabulary from tokenized outputs.
    """
    i = index
    for output in outputs:
        tokens = tokenize_output(output)
        for token in tokens:
            if token not in vocab:
                vocab[token] = i
                i += 1
    return vocab, i
def encode_outputs(outputs, vocab):
    encoded = []
    for output in outputs:
        tokens = tokenize_output(output)  # Tokenize the output
        sequence = [vocab.get(token, vocab.get("<UNK>", 0)) for token in tokens] # [vocab.get("<SOS>", 0)] + \+ \[vocab.get("<EOS>", 0)]
        encoded.append(sequence)
    return encoded

def pad_sequences_to_fixed_length(sequences, max_len):
    """
    Pads sequences to a fixed length.
    """
    return pad_sequences(sequences, maxlen=max_len, padding="post", value=vocab["<PAD>"])

def decode_sequence(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    tokens = [inv_vocab[idx] for idx in sequence if idx in inv_vocab and idx not in { vocab["<SOS>"], vocab["<EOS>"],vocab["<PAD>"]} ] 
    output = " ".join(tokens)
    output = output.replace(" ( ", " (").replace("( ", "(") #.replace(" )", ")")
    return output

def decode_sequence_2(sequence, vocab):
    """
    Decodes a sequence of integers back into the structured output string.
    """
    inv_vocab = {v: k for k, v in vocab.items()}  # Reverse the vocabulary
    # sequence = sequence.cpu().tolist()  # Convert tensor to a list of integers
    tokens = [inv_vocab.get(idx, "") for idx in sequence if idx > 0]  # Ignore unknown and put empty char
    return "".join(tokens)



In [11]:


def prepare_data(
    X_train, y_train, X_test, y_test, max_len_1=20, max_len_2 = 20
):

    index = 4
    X_vocab, index = build_vocab(X_train,index)  # Build vocabulary from training outputs
    X_train_encoded = encode_outputs(X_train, X_vocab)  # Encode training outputs
    X_test_encoded = encode_outputs(X_test, X_vocab)  # Encode testing outputs
    X_train_processed = pad_sequences_to_fixed_length(X_train_encoded, max_len_1)
    X_test_processed = pad_sequences_to_fixed_length(X_test_encoded, max_len_1)

    y_train_processed = pad_sequences_to_fixed_length(y_train, max_len_2)
    y_test_processed = pad_sequences_to_fixed_length(y_test, max_len_2)


    return (
        X_train_processed,
        X_test_processed,
        y_train_processed,
        y_test_processed,
        X_vocab,
    )


In [12]:
X_train_processed, X_test_processed, y_train_processed, y_test_processed, vocab  = prepare_data( X_train, y_train, X_test, y_test, max_len_1=40, max_len_2=40)

In [13]:
X_train_processed

array([[  4,   5,   6, ...,   0,   0,   0],
       [ 12,  13,  14, ...,   0,   0,   0],
       [ 18,  19,  20, ...,   0,   0,   0],
       ...,
       [ 48, 517, 512, ...,   0,   0,   0],
       [508,  20,   4, ...,   0,   0,   0],
       [508,  26,  22, ...,   0,   0,   0]])

In [14]:
y_train_processed[1]

array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 'large': 4,
 'pie': 5,
 'with': 6,
 'green': 7,
 'pepper': 8,
 'and': 9,
 'extra': 10,
 'ham': 11,
 'party': 12,
 'size': 13,
 'stuffed': 14,
 'crust': 15,
 'artichokes': 16,
 'mushroom': 17,
 'i': 18,
 'want': 19,
 'one': 20,
 'regular': 21,
 'pizza': 22,
 'without': 23,
 'any': 24,
 'basil': 25,
 'a': 26,
 'american': 27,
 'cheese': 28,
 'little': 29,
 'bit': 30,
 'of': 31,
 'sausage': 32,
 "'d": 33,
 'like': 34,
 'sized': 35,
 'high': 36,
 'rise': 37,
 'dough': 38,
 'lot': 39,
 'banana': 40,
 'chicken': 41,
 'black': 42,
 'olives': 43,
 'sauce': 44,
 'broccoli': 45,
 'peperonni': 46,
 'italian': 47,
 'can': 48,
 'have': 49,
 'flatbread': 50,
 'style': 51,
 'lunch': 52,
 '-': 53,
 'blue': 54,
 'need': 55,
 'caramelized': 56,
 'onions': 57,
 'combination': 58,
 'eggplant': 59,
 'pecorino': 60,
 'New': 61,
 'York': 62,
 'artichoke': 63,
 'spinach': 64,
 'Neapolitan': 65,
 'bacon': 66,
 'tofu': 67,
 'grilled': 68,
 'mozzarella': 69,
 

In [16]:
X_test_processed

array([[ 18,  19,  26, ...,   0,   0,   0],
       [ 18, 513,  34, ...,   0,   0,   0],
       [ 48,  18, 512, ...,   0,   0,   0],
       ...,
       [ 18, 509, 519, ...,   0,   0,   0],
       [ 18, 597,  20, ...,   0,   0,   0],
       [ 18,  33,  34, ...,   0,   0,   0]])

In [17]:
y_test_processed

array([[5, 5, 1, ..., 0, 0, 0],
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 5, ..., 0, 0, 0],
       ...,
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 1, ..., 0, 0, 0],
       [5, 5, 5, ..., 0, 0, 0]])

In [18]:
X_test_processed.shape, y_test_processed.shape


((1357, 40), (1357, 40))

In [19]:
X_train_processed.shape, y_train_processed.shape

((1871720, 40), (1871720, 40))

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.long) 
        self.targets = torch.tensor(targets, dtype=torch.long) 

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "src_input_ids": self.inputs[idx],
            "tgt_input_ids": self.targets[idx],
        }



train_dataset = SequenceDataset(X_train_processed, y_train_processed)
test_dataset = SequenceDataset(X_test_processed, y_test_processed)

batch_size = 128  # Adjust based GPU ;-;  memory
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import BatchNorm1d
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=vocab["<PAD>"])
        self.bilstm_1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout,
        )

        self.fc2 = nn.Linear(hidden_dim * 2, output_dim)


    def forward(self, x):
        embedded = self.embedding(x)

        lstm_out, _ = self.bilstm_1(embedded)

        output = self.fc2(lstm_out)
        return output


In [22]:
len(vocab)

607

In [23]:
max(vocab.values())

606

In [24]:
input_dim = len(vocab)
embedding_dim = 128
hidden_dim = 128  
output_dim = 6
num_layers = 2  
dropout = 0.3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #"cuda" if torch.cuda.is_available() else
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)

In [25]:
def evaluate_model_with_accuracy(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    total_tokens = 0
    correct_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output_dim = output.shape[-1]

            # Flatten outputs and targets
            output = output.view(-1, output_dim)  # Shape: (batch_size * seq_len, output_dim)
            tgt = tgt.view(-1)  # Shape: (batch_size * seq_len)

            # Apply mask to remove padding tokens
            mask = tgt != 0  # Mask to ignore padding indices
            output = output[mask]  # Filter model outputs
            tgt = tgt[mask]  # Filter targets

            # Compute loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

            # Calculate accuracy
            predictions = output.argmax(dim=1)  # Get the index of the max log-probability
            valid_indices = tgt != 0
            correct_tokens += (predictions[valid_indices] == tgt[valid_indices]).sum().item()
            total_tokens += valid_indices.sum().item()

    accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
    return epoch_loss / len(dataloader), accuracy


In [26]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss(ignore_index=0)  # Use for multi-class classification ignore_index=0 for padding
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

for epoch in range(20):  # Number of epochs
    model.train()
    epoch_loss = 0
    total_batches = len(train_dataloader)
    progress_bar = tqdm(train_dataloader, desc="Training Progress", unit="batch", leave=True)

    for batch_idx, batch in enumerate(progress_bar):  # Assuming a DataLoader is used
        src = batch["src_input_ids"].to(device)  # Input tokens
        tgt = batch["tgt_input_ids"].to(device)  # Target tokens

        optimizer.zero_grad()
        output = model(src)  # Forward pass
        output_dim = output.shape[-1]

        # Flatten outputs and targets for loss computation
        output = output.view(-1, output_dim)  # Shape: (batch_size * seq_len, output_dim)
        tgt = tgt.view(-1)  # Shape: (batch_size * seq_len)

        # Compute loss
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        avg_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_description(f"Training Progress: Batch {batch_idx + 1}/{total_batches}, Avg Loss: {avg_loss:.8f}")

    val_loss, accuracy = evaluate_model_with_accuracy(model, test_dataloader, criterion, device)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_dataloader):.8f}, Val Loss: {val_loss:.8f}, Accuracy: {accuracy * 100:.4f}%")


Training Progress: Batch 14623/14623, Avg Loss: 0.01478765: 100%|██████████| 14623/14623 [03:56<00:00, 61.73batch/s]


Epoch 1, Loss: 0.01478765, Val Loss: 0.47213051, Accuracy: 87.1107%


Training Progress: Batch 14623/14623, Avg Loss: 0.00891087: 100%|██████████| 14623/14623 [04:01<00:00, 60.43batch/s]


Epoch 2, Loss: 0.00891087, Val Loss: 0.43002506, Accuracy: 87.8602%


Training Progress: Batch 14623/14623, Avg Loss: 0.00838830: 100%|██████████| 14623/14623 [04:09<00:00, 58.60batch/s]


Epoch 3, Loss: 0.00838830, Val Loss: 0.46082289, Accuracy: 87.8127%


Training Progress: Batch 14623/14623, Avg Loss: 0.00803249: 100%|██████████| 14623/14623 [04:08<00:00, 58.82batch/s]


Epoch 4, Loss: 0.00803249, Val Loss: 0.45537149, Accuracy: 86.5249%


Training Progress: Batch 3357/14623, Avg Loss: 0.00777046:  23%|██▎       | 3357/14623 [00:56<03:11, 58.94batch/s]


KeyboardInterrupt: 

In [27]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

In [28]:
save_model(model, "../weights/Bilstm_order_sequence.pt")

Model saved to ../weights/Bilstm_order_sequence.pt


In [29]:
model = BiLSTMModel(input_dim, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
model = load_model(model,"../weights/Bilstm_order_sequence.pt")

Model loaded from ../weights/Bilstm_order_sequence.pt


C:\Users\yousi\AppData\Local\Temp\ipykernel_11300\1876629435.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


In [30]:
sequence = test_dataset.__getitem__(0)["src_input_ids"].cpu().tolist()
pred_sequence= decode_sequence( sequence,vocab)
pred_sequence

'i want a pizza with pesto and mushrooms but no pineapple'

In [31]:

with torch.no_grad():
    output = model( test_dataset.__getitem__(0)["src_input_ids"].to(device))
    predictions = output.argmax(dim=1)
predictions

tensor([5, 5, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], device='cuda:0')

In [32]:
tgt= test_dataset.__getitem__(0)["tgt_input_ids"].to(device)
tgt

tensor([5, 5, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [33]:
predictions == tgt

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False],
       device='cuda:0')

In [34]:
def evaluate_model_with_sequence_accuracy(model, dataloader, device):
    model.eval()
    total_sequences = 0
    correct_sequences = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src)
            output = output.argmax(dim=-1)


            for pred, tgt_seq in zip(output, tgt):
                valid_mask = tgt_seq != 0  # Ignore padding in comparison
                pred = pred[valid_mask]
                tgt_seq = tgt_seq[valid_mask]
                
                if torch.equal(pred, tgt_seq): 
                    correct_sequences += 1
                else:
                    print(X_test[total_sequences])
                    print("Predicted:", pred)
                    print("Target:", tgt_seq)
                total_sequences += 1

    print(f"Correct {correct_sequences}, Total {total_sequences}")
    sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
    return sequence_accuracy * 100

In [35]:
evaluate_model_with_sequence_accuracy(model, test_dataloader, device)

can i get one small pepperoni and chicken pizza with no mushrooms
Predicted: tensor([5, 5, 5, 1, 2, 5, 5, 5, 2, 2, 2, 2], device='cuda:0')
Target: tensor([5, 5, 5, 1, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
get me two pepsis a coke and five large fantas
Predicted: tensor([5, 5, 3, 4, 4, 4, 5, 3, 4, 4], device='cuda:0')
Target: tensor([5, 5, 3, 4, 3, 4, 5, 3, 4, 4], device='cuda:0')
i'll go for one pepsi six large diet cokes and a medium fanta
Predicted: tensor([5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 5, 3, 4], device='cuda:0')
Target: tensor([5, 5, 5, 3, 4, 3, 4, 4, 4, 5, 3, 4, 4], device='cuda:0')
i would like one pie with sausage olives and pineapple
Predicted: tensor([5, 5, 1, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
Target: tensor([5, 5, 5, 1, 2, 2, 2, 2, 2, 2], device='cuda:0')
i want a medium chicken and tuna pizza with no pesto
Predicted: tensor([5, 5, 1, 2, 2, 5, 2, 2, 2, 2, 2], device='cuda:0')
Target: tensor([5, 5, 1, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
i need two medium pepperon

47.60501105379514